In [ ]:
import nltk
import requests
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
NEWS_API_KEY = "60697fb4390d400b83ef934d1c2d0dc1"

In [ ]:
def fetch_news():
    url = f"https://newsapi.org/v2/everything?q=geopolitics&language=en&pageSize=10&sortBy=publishedAt&apiKey={NEWS_API_KEY}"
    response=requests.get(url)
    data=response.json()
    articles=data.get("articles",[])
    news = [article["title"] + ". " + str(article["description"]) for article in articles]
    return news


In [ ]:


def analyze_sentiment(news_list):
    results=[]
    for news in news_list:
        sentiment=sia.polarity_scores(news)
        results.append({
            "news":news,
            "sentiment_score":sentiment["compound"]
        })
    return results

In [ ]:
news=fetch_news()
sentiment_result=analyze_sentiment(news)

for item in sentiment_result:
    print(f"##{item['news']}\n  -> Sentiment Score: {item['sentiment_score']}\n")
    

In [57]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("gem_key")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.6)

In [60]:
def get_llm_relevance(news_text):
    prompt=f"Is the following news article likely to influence stock markets?If yes,state yes and explain how.\n\nArticle:{news_text}"
    response=llm.invoke(prompt)
    return response.content.strip()

In [61]:
def filter(news_list):
    relevent_articles=[]
    for article in news_list:
        relevance_score=get_llm_relevance(article)
        if "yes" in relevance_score.lower():
            relevent_articles.append(article)
    return relevent_articles